2023/12
## skku DataMining project : 
## 서울 대중교통 혼잡도 및 중심성 그래프분석(Seoul_transportation_GraphMining)
contributer : 김민, 김효원, 이수정, 조병웅  
instruction :   
블로거 GD park manger님의 대중교통 분석 연구를 바탕으로, 서울시 대중교통 네트워크의 중심성을 살펴보고,
사용목적 대비 유동인구 수라는 혼잡도 개념을 추가하여 새로 발전된 인사이트를 제공함을 목적으로 함.
참고한 분석 연구의 주소 -> https://m.blog.naver.com/gdpresent/221147909255

## graph_combine 파일 정보
contributer : 조병웅, 김효원
두 개의 그래프, 버스 그래프와 지하철 그래프를 병합하여 하나의 그래프로 만드는 코드

## 라이브러리 및 데이터 로딩

In [1]:
import pandas as pd
import networkx as nx

In [2]:
node_subway = pd.read_csv('../데이터/전처리1_생성데이터/node_subway.csv', encoding='cp949')
node_bus = pd.read_csv('../데이터/전처리1_생성데이터/node_bus.csv', encoding='cp949')
edge_subway = pd.read_csv('../데이터/전처리1_생성데이터/edge_subway.csv', encoding='cp949')
edge_bus = pd.read_csv('../데이터/전처리1_생성데이터/edge_bus.csv', encoding='cp949')

## 전처리

In [3]:
# 중복 제거 및 병합
node_subway['type'] = 'subway'
node_bus['type'] = 'bus'
node = pd.concat([node_subway, node_bus], ignore_index=True)
node.drop_duplicates(subset=['title'], inplace=True) # title 기준 중복 제거
node.sort_values(by=['ARS_ID'], inplace=True)
node

,title,ARS_ID,name,X,Y,type
0,1_4.19민주묘지역,1,4.19민주묘지역,127.013689,37.649527,subway
1,2_가능역,2,가능역,127.044275,37.748393,subway
2,3_가락시장역,3,가락시장역,127.118258,37.493092,subway
3,4_가산디지털단지역,4,가산디지털단지역,126.882644,37.480389,subway
4,5_가양역,5,가양역,126.854402,37.561429,subway
...,...,...,...,...,...,...
896,68270_광사1통마을회관,68270,광사1통마을회관,127.069836,37.788065,bus
920,68271_광사1통마을회관,68271,광사1통마을회관,127.070235,37.788093,bus
870,68290_덕정주공6단지,68290,덕정주공6단지,127.073694,37.837017,bus
885,68351_삼숭교차로,68351,삼숭교차로,127.103922,37.810362,bus


In [4]:
edge_add = edge_subway.copy()
edge_add['source'], edge_add['target'] = edge_subway['target'], edge_subway['source']
edge_add

,LaneNo,source,target
0,1,89_덕계역,90_덕정역
1,1,272_용산역,63_남영역
2,1,68_노량진역,272_용산역
3,1,84_대방역,68_노량진역
4,1,210_신길역,84_대방역
...,...,...,...
431,113,6_가오리역,348_화계역
432,113,1_4.19민주묘지역,6_가오리역
433,113,194_솔밭공원역,1_4.19민주묘지역
434,113,153_북한산우이역,194_솔밭공원역


In [5]:
edge_subway['type'] = 'subway'
edge_add['type'] = 'subway'
edge_bus['type'] = 'bus'

edge = pd.concat([edge_subway, edge_add, edge_bus], ignore_index=True)
edge

,LaneNo,source,target,type
0,1,90_덕정역,89_덕계역,subway
1,1,63_남영역,272_용산역,subway
2,1,272_용산역,68_노량진역,subway
3,1,68_노량진역,84_대방역,subway
4,1,84_대방역,210_신길역,subway
...,...,...,...,...
14272,775,63240_교육지원청,63242_파주시청,bus
14273,775,63242_파주시청,63233_장안아파트,bus
14274,775,63233_장안아파트,63201_금촌사거리,bus
14275,775,63201_금촌사거리,63213_장안미래아파트,bus


In [6]:
node['title'] = node['title'].str.replace(' ', '')
node['name'] = node['name'].str.replace(' ', '')
edge['source'] = edge['source'].str.replace(' ', '')
edge['target'] = edge['target'].str.replace(' ', '')

## 시각화 및 파일 저장

In [7]:
graph = nx.DiGraph()
graph.add_nodes_from(node.set_index('title').T.to_dict().items())
graph.add_edges_from(edge[['source', 'target']].values.tolist())
print(graph.number_of_nodes(), graph.number_of_edges())

4981 7167


In [8]:
# 파일 저장
node.to_csv('./node.csv', index=None, encoding='cp949')
edge.to_csv('./edge.csv', index=None, encoding='cp949')
nx.write_graphml_lxml(graph, 'graph.graphml')